---
# **PREPROCESSING**
---

## Importer les libraries et le dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, LassoCV

In [ ]:
data = pd.read_csv("C:/Users/zefer/OneDrive/Bureau/Master II/Machine Learning/Exam/credit.csv")

data.head()

## Conversion de format

In [ ]:
# Convertir les types de variables au bon format

data['Own'] = data['Own'].astype('category')
data['Student'] = data['Student'].astype('category')
data['Married'] = data['Married'].astype('category')
data['Region'] = data['Region'].astype('category')

In [ ]:
print(data['Own'].value_counts())
print(data['Student'].value_counts())
print(data['Married'].value_counts())
print(data['Region'].value_counts())

## Encodage des variables categorielles 

In [ ]:
# Encodage des variables categorielles 

data['Own'].replace(['Yes', 'No'], [0,1], inplace=True)
data['Student'].replace(['Yes', 'No'], [0,1], inplace=True)
data['Married'].replace(['Yes', 'No'], [0,1], inplace=True)
data['Region'].replace(['South', 'West', 'East'], [0,1, 2], inplace=True)

In [ ]:
data.head()

---
# 1) Fonction regression(X, Y) qui renvoie l’estimateur des moindre carrés.
---

In [ ]:
# Affectation des variables explicatives a X et la variablle endogene a y et redimensionnement sous forme matricielle

Y = data['Balance']
X = data.drop('Balance', axis=1)

$\hat{\beta} = (X^{T}X)^{-1}X^{T}X y$

In [ ]:
def regression(X, Y):
    X = pd.concat([X, pd.DataFrame(np.ones((X.shape[0],1)))], axis=1)
    XTX = X.T.dot(X)
    beta = np.linalg.pinv(XTX).dot(X.T).dot(Y)
    return beta

In [ ]:
regression(X, Y)

## Comparaison avec un modele lineaire classique 

In [ ]:
# Entrainement du modele
lr = LinearRegression()
lr.fit(X, Y)

#Intercept and Coefficient

beta = lr.intercept_
alpha = lr.coef_

In [ ]:
print("Interception est ", beta)
print("Les coefficients sont ", alpha)

---
# 2. Fonction regress(X, α, β) qui renvoie le vecteur Y-chapeau
---

$\hat{y_i}=(\alpha, x_i)+\beta$

In [ ]:
def regress(X, alpha, beta):
    Y_chapeau = np.array(X.dot(alpha.T)+beta)
    return Y_chapeau

In [ ]:
regress(X, alpha, beta)

---
# 3. Calculer l’erreur au sens des moindres carrés du régresseur
---

$\hat{errors} = \sum_{i=1}^{n}(y_i-f(x_i))^2$

In [ ]:
# On récupère l'erreur de norme 2 sur le jeu de données test comme baseline
error = np.sum((lr.predict(X) - Y) ** 2)

print(error)

#ou

Y_chapeau = np.array(X.dot(alpha.T)+beta)

def error(Y, Y_chapeau):
    return np.sum((Y - Y_chapeau)**2) # mean ou sum

error(Y, Y_chapeau)

---
# 4.
---

## a) Fonction ridge_regression(X, Y , lambda) qui renvoie l’estimateur des moindre carrés généralisés

$(\frac{\hat{\alpha}}{\hat{\beta}})=(X^{T}X+\lambda I_{p+1})^{-1}X^{T}Y$

In [ ]:
I = np.identity(X.shape[1]) # Matrice identite d'ordre p+1

def ridge_regression(X,Y,lambda_):
    return np.dot(np.dot(np.linalg.pinv(np.dot(X.T,X)+(lambda_*I)), X.T), Y)

In [ ]:
lambda_ = 1
ridge_regression(X, Y, lambda_)

## Fonction de ridge_regression classique

In [ ]:
rm = linear_model.Ridge()
rm.fit(X,Y)
print(rm.coef_)
print(rm.intercept_)

**Interpretation**:

 * On remarque que, tous les parametres de la regression ridge sont inferieur aux attributs(coef_ et intercep) d'un regresseur ridge de type classique 
 
*On peut donc conclure que la valeur de lambda diminue les poids de tous les paramètres de la régression ridge*

## b) Tracez l’évolution des coefficients du vecteur alpha en fonction du paramètre de régularisation lambda pour des valeurs entre 0.001 et 1000

In [ ]:
n_alphas = 200
alphas = np.logspace(-2, 3, n_alphas)

    
coefs = []
for l_ in alphas:  
    ridge = linear_model.Ridge(alpha=l_, fit_intercept=False)
    ridge.fit(X, Y)
    coefs.append(ridge.coef_)
    
coefs = np.array(coefs) 
coefs = coefs.reshape(coefs.shape[0], -1)



ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('lambda')
plt.ylabel('coefficients')
plt.title('Ridge coefficients as a function of the regularization')
plt.axis('tight')
plt.legend(labels = [])
plt.show()

###  variables semblent le mieux expliquer la variable Balance ?

In [ ]:
import seaborn as sns

corr = d.corr()
#Plot figsize
fig, ax = plt.subplots(figsize=(8, 8))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, cmap='RdBu', annot=True, fmt=".2f")
#Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
#Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
#show plot
plt.show()

**Interpretation :**

 * Les variables qui expliquent mieux la variable *Balance* sont : *Income, Limit, Cards, Age et Education*

## c) 

## - La meilleure valeur pour le paramètre lambda

*La meilleur de alpha est lorsque il est le plus bas, c'est a dire alpha minimal*

In [ ]:
regr_cv = RidgeCV(alphas)

# Entrainement du model
model_cv = regr_cv.fit(X, Y) 

# alpha min
model_cv.alpha_

## - Calculons l’erreur au sens des moindres carrés

In [ ]:
Y_chapeau = model_cv.predict(X) 
ridge_error = error(Y, Y_chapeau)
ridge_error

 * L'erreur de la regression ridge est la meme que l'erreur au sens des moindres du regresseur

---
# 5
---

## a) En utilisant la classe linear_model.Lasso, tracez l’évolution des coefficients du vecteur αˆ en fonction de la valeur du paramètre lambda

In [ ]:
n_alphas = 300
alphas = np.logspace(-5, 1, n_alphas)
lasso = linear_model.Lasso(fit_intercept=False)

coefs = []
errors = []
for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X, Y)
    coefs.append(lasso.coef_)
    errors.append([error, np.mean((lasso.predict(X) - Y) ** 2)])


ax = plt.gca()

ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('Lambda')
plt.ylabel('weights')
plt.axis('tight')
plt.show()

 * Les variables qui expliquent mieux la variable *Balance* sont : *Income, Limit, Cards, Age et Education*

 * Elle sont les memes que celle trouvee a la question precedente

## b) 

## - La meilleure valeur pour le paramètre lambda

In [ ]:
regr_lasso = LassoCV(cv = 20 )

# Entrement du modele
model_lasso = regr_lasso.fit(X, Y) 

# Valeur d'alpha
model_lasso.alpha_ 

**Interpretation :**

 * Comme on peut le voir, le lasso permet de supprimer des variables en mettant leur poids à zéro. C'est le cas si deux variables sont corrélées. L'une sera sélectionnée par le Lasso, l'autre supprimée

# 6. Choix du modele approprie


 * Dans les deux modeles( Ridge et Lasso), la régularisation fonctionne bien, et permet de réduire l'erreur totale du modèle à l'aide de l'ajout du biais qui quantifie la complexité. 
 
    * la regression Ridge diminue grandement l'influence de certaines variables sur le modèle
    
    * la regression Lasso peut directement supprimer l'influence de certaines variables sur le modele (mettre leur poids à zéro). 

*Les erreurs sont tres reduites dans la regression Lasso. Donc le Lasso est plus efficace, parcissiomieuse et approprie que le Ridge*
    